## Benchmarking using SemHash on NLU Evaluation Corpora

This notebook benchmarks the results on the 3 NLU Evaluation Corpora:
1. Ask Ubuntu Corpus
2. Chatbot Corpus
3. Web Application Corpus


More information about the dataset is available here: 

https://github.com/sebischair/NLU-Evaluation-Corpora


* Semantic Hashing is used as a featurizer. The idea is taken from the paper:

https://www.microsoft.com/en-us/research/publication/learning-deep-structured-semantic-models-for-web-search-using-clickthrough-data/

* Benchmarks are performed on the same train and test datasets used by the other benchmarks performed in the past. One important paper that benchmarks the datasets mentioned above on some important platforms (Dialogflow, Luis, Watson and RASA) is : 

http://workshop.colips.org/wochat/@sigdial2017/documents/SIGDIAL22.pdf

* Furthermore, Botfuel made another benchmarks with more platforms (Recast, Snips and their own) and results can be found here: 

https://github.com/Botfuel/benchmark-nlp-2018

* The blogposts about the benchmarks done in the past are available at : 

https://medium.com/botfuel/benchmarking-intent-classification-services-june-2018-eb8684a1e55f

https://medium.com/snips-ai/an-introduction-to-snips-nlu-the-open-source-library-behind-snips-embedded-voice-platform-b12b1a60a41a

* To be very fair on our benchmarks and results, we used the same train and test set used by the other benchmarks and no cross validation or stratified splits were used. The test data was not used in any way to improve the results. The dataset used can be found here:

https://github.com/Botfuel/benchmark-nlp-2018/tree/master/results



In [2]:
from __future__ import unicode_literals
import re
import os
import codecs
import json
import csv
import spacy
import sklearn
import numpy as np
import matplotlib.pyplot as plt
from sklearn import model_selection
from time import time
from sklearn.feature_extraction.text import TfidfVectorizer, HashingVectorizer, CountVectorizer
from sklearn.feature_selection import SelectFromModel, SelectKBest, chi2
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.neighbors.nearest_centroid import NearestCentroid
from sklearn.linear_model import RidgeClassifier
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import Perceptron
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestCentroid
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.utils.extmath import density
from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
import numpy as np

Spacy english dataset needs to be present. It can be downloaded using the following command:

python -m spacy download en

In [3]:
nlp=spacy.load('en_core_web_lg')

In [4]:
intent_dict_AskUbuntu = {"Make Update":0, "Setup Printer":1, "Shutdown Computer":2, "Software Recommendation":3, "None":4}
intent_dict_Chatbot = {"DepartureTime":0, "FindConnection":1}
intent_dict_WebApplications = {"Download Video":0, "Change Password":1, "None":2, "Export Data":3, "Sync Accounts":4,
                  "Filter Spam":5, "Find Alternative":6, "Delete Account":7}

In [5]:
benchmark_dataset = 'WebApplications' #choose from 'AskUbuntu', 'Chatbot' or 'WebApplications'

In [6]:
def read_CSV_datafile(filename):    
    X = []
    y = []
    with open(filename, 'r') as csvfile:
        reader = csv.reader(csvfile, delimiter='\t')
        for row in reader:
            X.append(row[0])
            if benchmark_dataset == 'AskUbuntu':
                y.append(intent_dict_AskUbuntu[row[1]])
            elif benchmark_dataset == 'Chatbot':
                y.append(intent_dict_Chatbot[row[1]])
            else:
                y.append(intent_dict_WebApplications[row[1]])           
    return X,y


In [8]:
filename_train = "./datasets/KL/WebApplication/train.csv"
filename_test= "./datasets/KL/WebApplication/test.csv"
X_train_raw, y_train_raw = read_CSV_datafile(filename = filename_train)
X_test_raw, y_test_raw = read_CSV_datafile(filename = filename_test)

In [9]:
print("Training data samples: \n",X_train_raw[-5:-1], "\n\n")

print("Class Labels: \n", y_train_raw[-5:-1], "\n\n")

print("Size of Training Data: {}".format(len(X_train_raw)))

Training data samples: 
 ['How do I delete my Gmail account?', 'How do I delete my Experts Exchange account?', 'How do I delete my Ohloh profile?', 'How can I permanently delete my MySpace account?'] 


Class Labels: 
 [7, 7, 7, 7] 


Size of Training Data: 30


In [10]:
def tokenize(doc):
    """
    Returns a list of strings containing each token in `sentence`
    """
    #return [i for i in re.split(r"([-.\"',:? !\$#@~()*&\^%;\[\]/\\\+<>\n=])",
    #                            doc) if i != '' and i != ' ' and i != '\n']
    tokens = []
    doc = nlp.tokenizer(doc)
    for token in doc:
        tokens.append(token.text)
    return tokens

In [11]:
def preprocess(doc):
    clean_tokens = []
    doc = nlp(doc)
    for token in doc:
        if not token.is_stop:
            clean_tokens.append(token.lemma_)
    return " ".join(clean_tokens)

# SemHash

In [12]:
def find_ngrams(input_list, n):
    return zip(*[input_list[i:] for i in range(n)])

def semhash_tokenizer(text):
    tokens = text.split(" ")
    final_tokens = []
    for unhashed_token in tokens:
        hashed_token = "#{}#".format(unhashed_token)
        final_tokens += [''.join(gram)
                         for gram in list(find_ngrams(list(hashed_token), 3))]
    return final_tokens

def semhash_corpus(corpus):
    new_corpus = []
    for sentence in corpus:
        sentence = preprocess(sentence)
        tokens = semhash_tokenizer(sentence)
        new_corpus.append(" ".join(map(str,tokens)))
    return new_corpus

X_train_raw = semhash_corpus(X_train_raw)
X_test_raw = semhash_corpus(X_test_raw)

In [13]:
def get_vectorizer(corpus, preprocessor=None, tokenizer=None):
    vectorizer = CountVectorizer(ngram_range=(2,4),analyzer='char')
    vectorizer.fit(corpus)
    return vectorizer, vectorizer.get_feature_names()

In [15]:
def trim(s):
    """Trim string to fit on terminal (assuming 80-column display)"""
    return s if len(s) <= 80 else s[:77] + "..."


# #############################################################################
# Benchmark classifiers
def benchmark(clf, X_train, y_train, X_test, y_test, target_names,
              print_report=True, feature_names=None, print_top10=False,
              print_cm=True):
    print('_' * 80)
    print("Training: ")
    print(clf)
    t0 = time()
    clf.fit(X_train, y_train)
    train_time = time() - t0
    print("train time: %0.3fs" % train_time)

    t0 = time()
    pred = clf.predict(X_test)
    test_time = time() - t0
    print("test time:  %0.3fs" % test_time)

    score = metrics.accuracy_score(y_test, pred)
    print("accuracy:   %0.3f" % score)
    #print("Accuracy: %0.3f (+/- %0.3f)" % (score.mean(), score.std() * 2))

    if hasattr(clf, 'coef_'):
        print("dimensionality: %d" % clf.coef_.shape[1])
        print("density: %f" % density(clf.coef_))

        if print_top10 and feature_names is not None:
            print("top 10 keywords per class:")
            for i, label in enumerate(["Make Update", "Setup Printer", "Shutdown Computer","Software Recommendation", "None"]):
                top10 = np.argsort(clf.coef_[i])[-10:]
                print(trim("%s: %s" % (label, " ".join([feature_names[i] for i in top10]))))
        print()

    if print_report:
        print("classification report:")
        print(metrics.classification_report(y_test, pred,
                                            target_names=target_names))

    if print_cm:
        print("confusion matrix:")
        print(metrics.confusion_matrix(y_test, pred))

    print()
    clf_descr = str(clf).split('(')[0]
    return clf_descr, score, train_time, test_time

In [16]:
def plot_results(results):
    # make some plots
    indices = np.arange(len(results))

    results = [[x[i] for x in results] for i in range(4)]

    clf_names, score, training_time, test_time = results
    training_time = np.array(training_time) / np.max(training_time)
    test_time = np.array(test_time) / np.max(test_time)

    plt.figure(figsize=(12, 8))
    plt.title("Score")
    plt.barh(indices, score, .2, label="score", color='navy')
    plt.barh(indices + .3, training_time, .2, label="training time",
             color='c')
    plt.barh(indices + .6, test_time, .2, label="test time", color='darkorange')
    plt.yticks(())
    plt.legend(loc='best')
    plt.subplots_adjust(left=.25)
    plt.subplots_adjust(top=.95)
    plt.subplots_adjust(bottom=.05)

    for i, c in zip(indices, clf_names):
        plt.text(-.3, i, c)
    plt.savefig("./results_AskUbuntu.png", format="png")
    plt.show()

In [17]:
def data_for_training():
    vectorizer, feature_names = get_vectorizer(X_train_raw, preprocessor=preprocess, tokenizer=tokenize)
    
    X_train_no_HD = vectorizer.transform(X_train_raw).toarray()
    X_test_no_HD = vectorizer.transform(X_test_raw).toarray()
            
    return X_train_no_HD, y_train_raw, X_test_no_HD, y_test_raw, feature_names



In [18]:
X_train, y_train, X_test, y_test, feature_names = data_for_training()

In [19]:
#     X_train, y_train, X_test, y_test, feature_names = data_for_training()
def ngram_encode(str_test, HD_aphabet, aphabet, n_size): # method for mapping n-gram statistics of a word to an N-dimensional HD vector
    HD_ngram = np.zeros(HD_aphabet.shape[1]) # will store n-gram statistics mapped to HD vector
    full_str = '#' + str_test + '#' # include extra symbols to the string
        
    for il, l in enumerate(full_str[:-(n_size-1)]): # loops through all n-grams
        hdgram = HD_aphabet[aphabet.find(full_str[il]), :] # picks HD vector for the first symbol in the current n-gram
        for ng in range(1, n_size): #loops through the rest of symbols in the current n-gram
            hdgram = hdgram * np.roll(HD_aphabet[aphabet.find(full_str[il+ng]), :], ng) # two operations simultaneously; binding via elementvise multiplication; rotation via cyclic shift
            
        HD_ngram += hdgram # increments HD vector of n-gram statistics with the HD vector for the currently observed n-gram
    
    HD_ngram_norm = np.sqrt(HD_aphabet.shape[1]) * (HD_ngram/ np.linalg.norm(HD_ngram) )  # normalizes HD-vector so that its norm equals sqrt(N)       
    return HD_ngram_norm # output normalized HD mapping



N = 1000 # set the desired dimensionality of HD vectors
n_size=3 # n-gram size
aphabet = 'abcdefghijklmnopqrstuvwxyz#' #fix the alphabet. Note, we assume that capital letters are not in use 
np.random.seed(1) # for reproducibility
HD_aphabet = 2 * (np.random.randn(len(aphabet), N) < 0) - 1 # generates bipolar {-1, +1}^N HD vectors; one random HD vector per symbol in the alphabet

# str='High like a basketball jump' # example string to represent using n-grams

# print(len(ngram_encode(str, HD_aphabet, aphabet, n_size))) # HD_ngram is a projection of n-gram statistics for str to N-dimensional space. It can be used to learn the word embedding
# print(X_train_raw[:1])

for i in range(len(X_train_raw)):
     X_train_raw[i] = ngram_encode(X_train_raw[i], HD_aphabet, aphabet, n_size) # HD_ngram is a projection of n-gram statistics for str to N-dimensional space. It can be used to learn the word embedding
# print(X_train_raw[:5])
for i in range(len(X_test_raw)):
    X_test_raw[i] = ngram_encode(X_test_raw[i], HD_aphabet, aphabet, n_size)
# print(X_test_raw[:5])

X_train, y_train, X_test, y_test = X_train_raw, y_train_raw, X_test_raw, y_test_raw
# print(X_train[:5])


In [20]:
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [34]:
set(y_train)

{0, 1, 2, 3, 4, 5, 6, 7}

In [35]:
count = {}
for i in range(len(y_train)):
    try:
        count[target_names[y_train[i]]]+=1
    except:
        count[target_names[y_train[i]]] = 1
print(str(count))


{'Download Video': 1, 'Change Password': 2, 'None': 2, 'Export Data': 2, 'Sync Accounts': 3, 'Filter Spam': 6, 'Find Alternative': 7, 'Delete Account': 7}


In [29]:
target_names = list(intent_dict_WebApplications.keys())

In [30]:
target_names

['Download Video',
 'Change Password',
 'None',
 'Export Data',
 'Sync Accounts',
 'Filter Spam',
 'Find Alternative',
 'Delete Account']

In [36]:
import matplotlib.pyplot as plt
for i_s, split in enumerate(range(1)):
    print("Evaluating Split {}".format(i_s))
#     X_train, y_train, X_test, y_test, feature_names = data_for_training()
    #target_names = ["Make Update", "Setup Printer", "Shutdown Computer","Software Recommendation", "None"]
    print("Train Size: {}\nTest Size: {}".format(X_train.shape[0], X_test.shape[0]))
    results = []
    #alphas = np.array([1,0.1,0.01,0.001,0.0001,0])
    parameters_mlp={'hidden_layer_sizes':[(100,50), (300, 100),(300,200,100)]}
    parameters_RF={ "n_estimators" : [50,60,70],
           "min_samples_leaf" : [1, 11]}
    k_range = list(range(3,7))
    parameters_knn = {'n_neighbors':k_range}
    knn=KNeighborsClassifier(n_neighbors=5)
    for clf, name in [  
#             (RidgeClassifier(tol=1e-2, solver="lsqr"), "Ridge Classifier"),
#             (GridSearchCV(knn,parameters_knn, cv=5),"gridsearchknn"),
#             #(Perceptron(n_iter=50), "Perceptron"),
#             (GridSearchCV(MLPClassifier(activation='tanh'),parameters_mlp, cv=5),"gridsearchmlp"),
#            # (MLPClassifier(hidden_layer_sizes=(100, 50), activation="logistic", max_iter=300), "MLP"),
#             #(MLPClassifier(hidden_layer_sizes=(300, 100, 50), activation="logistic", max_iter=500), "MLP"),
#            # (MLPClassifier(hidden_layer_sizes=(300, 100, 50), activation="tanh", max_iter=500), "MLP"),
            (PassiveAggressiveClassifier(100), "Passive-Aggressive"),
#            # (KNeighborsClassifier(n_neighbors=1), "kNN"),
#            # (KNeighborsClassifier(n_neighbors=3), "kNN"),
#            # (KNeighborsClassifier(n_neighbors=5), "kNN"),
#             #(KNeighborsClassifier(n_neighbors=10), "kNN"),
#             (GridSearchCV(RandomForestClassifier(n_estimators=10),parameters_RF, cv=5),"gridsearchRF")
#             #(RandomForestClassifier(n_estimators=10), "Random forest"),
            (RandomForestClassifier(n_estimators=50), "Random forest")
    ]:
           
#         print('=' * 80)
        print(name)
        results.append(benchmark(clf, X_train, y_train, X_test, y_test, target_names,
                                 feature_names=feature_names))

    print("LinearSVC with L1-based feature selection")
    plot_results(results)
    
    

Evaluating Split 0
Train Size: 30
Test Size: 59
Passive-Aggressive
________________________________________________________________________________
Training: 
PassiveAggressiveClassifier(C=100, average=False, class_weight=None,
                            early_stopping=False, fit_intercept=True,
                            loss='hinge', max_iter=1000, n_iter_no_change=5,
                            n_jobs=None, random_state=None, shuffle=True,
                            tol=0.001, validation_fraction=0.1, verbose=0,
                            warm_start=False)
train time: 0.014s
test time:  0.024s
accuracy:   0.797
dimensionality: 1000
density: 1.000000

classification report:
                  precision    recall  f1-score   support

  Download Video       0.00      0.00      0.00         0
 Change Password       1.00      0.67      0.80         6
            None       0.00      0.00      0.00         4
     Export Data       1.00      0.33      0.50         3
   Sync Accounts    

/home/suraj18025/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/suraj18025/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


ValueError: Number of classes, 7, does not match size of target_names, 8. Try specifying the labels parameter

In [ ]:
import time as timmer

def ngram_encode_mod(str_test, HD_aphabet, aphabet, n_size): # method for mapping n-gram statistics of a word to an N-dimensional HD vector
    HD_ngram = np.zeros(HD_aphabet.shape[1]) # will store n-gram statistics mapped to HD vector
    full_str = '#' + str_test + '#' # include extra symbols to the string
    shift=n_size-1
        
    # form the vector for the first n-gram
    hdgram = HD_aphabet[aphabet.find(full_str[0]), :] # picks HD vector for the first symbol in the current n-gram
#     for ng in range(1, n_size): #loops through the rest of symbols in the current n-gram
#         hdgram = hdgram * np.roll(HD_aphabet[aphabet.find(full_str[0+ng]), :], ng) # two operations simultaneously; binding via elementvise multiplication; rotation via cyclic shift        
#     HD_ngram += hdgram # increments HD vector of n-gram statistics with the HD vector for the currently observed n-gram
        
#     for l in range(1, len(full_str)-shift):  # form all other n-grams using the HD vector for the first one      
#         hdgram = np.roll( hdgram * HD_aphabet[aphabet.find(full_str[l-1]), :], -1) * np.roll(HD_aphabet[aphabet.find(full_str[l+shift]), :], shift) # improved implementation of forming HD vectors for n-grams
#         HD_ngram += hdgram # increments HD vector of n-gram statistics with the HD vector for the currently observed n-gram
    
#     HD_ngram_norm = np.sqrt(HD_aphabet.shape[1]) * (HD_ngram/ np.linalg.norm(HD_ngram) )  # normalizes HD-vector so that its norm equals sqrt(N)       
    
    HD_ngram_tot = np.zeros(HD_aphabet.shape[1])
    for n_size in range(2, 7):
        HD_ngram_tot += ngram_encode_hybrid(str_test, HD_aphabet, aphabet, n_size)
    HD_ngram_tot = np.sqrt(HD_aphabet.shape[1]) * (HD_ngram_tot/ np.linalg.norm(HD_ngram_tot))
    
#     return HD_ngram_norm # output normalized HD mapping
    return HD_ngram_tot


def ngram_encode_hybrid(str_test, HD_aphabet, aphabet, n_size): # method for mapping n-gram statistics of a word to an N-dimensional HD vector
    HD_ngram = np.zeros(HD_aphabet.shape[1]) # will store n-gram statistics mapped to HD vector
    full_str = '#' + str_test + '#' # include extra symbols to the string
    shift=n_size-1
#     print(full_str)
    
    if n_size < 3: # use the initial implementation
        #adjust the string for n-gram size
        if n_size == 1:
            full_str_e=full_str            
        else:
            full_str_e=full_str[:-(n_size-1)]    
            
        for il, l in enumerate(full_str_e): # loops through all n-grams
            hdgram = HD_aphabet[aphabet.find(full_str[il]), :] # picks HD vector for the first symbol in the current n-gram
            
            for ng in range(1, n_size): #loops through the rest of symbols in the current n-gram
                    hdgram = hdgram * np.roll(HD_aphabet[aphabet.find(full_str[il+ng]), :], ng) # two operations simultaneously; binding via elementvise multiplication; rotation via cyclic shift
    
            HD_ngram += hdgram # increments HD vector of n-gram statistics with the HD vector for the currently observed n-gram

    else:    # use the modified implementation

        # form the vector for the first n-gram
        hdgram = HD_aphabet[aphabet.find(full_str[0]), :] # picks HD vector for the first symbol in the current n-gram
        for ng in range(1, n_size): #loops through the rest of symbols in the current n-gram
            hdgram = hdgram * np.roll(HD_aphabet[aphabet.find(full_str[0+ng]), :], ng) # two operations simultaneously; binding via elementvise multiplication; rotation via cyclic shift        
        HD_ngram += hdgram # increments HD vector of n-gram statistics with the HD vector for the currently observed n-gram
            
        for l in range(1, len(full_str)-shift): # form all other n-grams using the HD vector for the first one    
            hdgram = np.roll( hdgram * HD_aphabet[aphabet.find(full_str[l-1]), :], -1) * np.roll(HD_aphabet[aphabet.find(full_str[l+shift]), :], shift) # improved implementation of forming HD vectors for n-grams
            HD_ngram += hdgram # increments HD vector of n-gram statistics with the HD vector for the currently observed n-gram
    
    HD_ngram_norm = np.sqrt(HD_aphabet.shape[1]) * (HD_ngram/ np.linalg.norm(HD_ngram) )  # normalizes HD-vector so that its norm equals sqrt(N)    
    
    return HD_ngram_norm # output normalized HD mapping



N_mod = 17000 # set the desired dimensionality of HD vectors
n_size_mod=3 # n-gram size
aphabet_mod = 'abcdefghijklmnopqrstuvwxyz# ?-/!\+[]{}().,:;1234567890äöÿã' #fix the alphabet. Note, we assume that capital letters are not in use 
# np.random.seed(1) # for reproducibility
HD_aphabet_mod = 2 * (np.random.randn(len(aphabet_mod), N_mod) < 0) - 1 # generates bipolar {-1, +1}^N HD vectors; one random HD vector per symbol in the alphabet

#str='jump' # example string to represent using n-grams
str_tst='intrigued started to work it out the third installment of the twilight saga eclipse proved to be a small stepup from the first two movies lowrie atoned for an error he made in the top half of the inning when he dropped a foul popup while playing first base but she couldnt sleep knowing that her mother was sitting alone somewhere in a big foreign airport that had gone into crisis mode call the minot at to reserve your spot for beginning duplicate bridge monday evenings plucking the track from her am sasha fierce beyonces halo has become one of her signature songs wow sound smart should change his name to really dumb it dabbles in several facets of the financial services industry from its jpmorgan investment banking chase credit cards and various banking operations the developing industrial applications for silver are exciting and expect longterm growth here rick morgan has been fishing lower manitou since he was a kid growing up in hibbing in the late he may not be a good guy but hes not a rapist at least not yet we have taken this request under consideration in gardner was facing trial for killing melvyn otterstrom a bartender in salt lake city when a girlfriend slipped him a gun at the courthouse soon after more offers came piling in from other schools central michigan miami western michigan and ohio for the current crop of condo dwellers losing the convenience of a car is part of the price of home ownership by values were in retreat will julian crocker look into this small problem but she said that officer didnt want to hear any more these were on increases in gross and average from their pathetic predecessors though neither measure was ever an accurate barometer of industry health the participants bacon cheeseburger patty with bacon lettuce tomato and mayo phillies catcher carlos ruiz connects for a gamewinning home run in the inning against the cardinals tango don cheadle is an undercover cop posing as a drug lord watching his criminal compatriot wesley snipes in a restrained return to form return to power after a lengthy prison stretch we are eliminating sales and management company cars altogether and moving those positions to an allowance said lisa kneggs fleet manager on a per school dayhourly basis has anyone done the math on this why dont we just pay everyone not to work peacock asks referring to losing the streak lot depends on the pitching staff of course but as the smoke cleared lost became as twitter fan joe hewitt put it a soap opera not an intellectual scifi thriller and thus a waste of time that is something hill hopes they all take seriously in worlds recent tests on mobile bandwidth speeds ranks first hands down its too early to judge whether a large number of donors across the country will pour money into the race as they did for brown but browns appearance here is likely to increase that possibility as the contest gains notice president ahmadinejad further announced irans decision to postpone talks with the world powers until august in a move to punish the west weir fire escape extension burley bridge mills viaduct road burley hes a house republican guy he may have been wearing number on his hip but tonights reebok boston indoor games at the reggie lewis center here was anything but unlucky for twotime olympic medallist bernard lagat adam vinatieri answered with a bomb of his own from yards many towns around the state are facing tightening budgets as certain services and repairs are left for another day whatrsquos amazing about jayz is that even nearing an age unheard of for a viable rapper even a decade ago the artist keeps putting out genredefying and essential albums is it rugged enough to handle outside terrain the war is coming to their cities the chechen rebel leader said in an interview on an islamist website he lasted only innings for his secondshortest outing of ford motor co has extended buyout offers once again to workers at its hamburg stamping plant a sign that the automaker is pushing hard to reduce its production work force rich barber says thats as it should be card companies can slap a penalty interest rate on existing balances if a customer falls at least days behind on payments obama then took on jay leno the nights entertainment saying great to see you jay the order was made through ovhs automated systems on the eve of retirement minnesota duluth chancellor kathryn martin seeks to finish her transformation of campus you are now over limit and so an over limit fee of rs can also be applied making the owed balance on the first page youll get photos of white folks being beaten up by groups of blacks and arabs in the spring mcdaniels made brandon marshall and tony scheffler who werent shy about expressing criticism his new jay cutlers its a sad day when a collegeuniversity honours a politician whose claim to fame was the businessoriented common sense revolution over someone anyone whose life serveds to inspire students to celebrate education the change while not material was accounted for on a retrospective basis and more closely aligns the depreciation policies with those of the companys drilling rigs which are depreciated based on operating days lets go to henry its popup menu lists all the pcs in your house that have been prepared for remote controlling including the kitchen laptop but there were other places to take a dip and a handful of bajans were taking advantage as dozed horn already had worked out for teams such as the carolina panthers and the philadelphia eagles but his times were in the mediocre to range notre dame beat providence next at syracuse saturday evolution as a search engine for commensurate energy and nutrient niches was now possible evolution as a euros the survival of the most fitting on this or any other wet rocky and sunlit planet and in any galaxy taking legal and critical measures of control in the sphere on nuclear security the international community should not ignore the global trends in energy and high technologies the judges did not goof in picking the medal winners mostly because the athletes last night made their jobs so easy ryan cross or potentially digby iaoni seem logical to me to add a bit of size what are your thoughts susan joy share holds one of her fish puppets from the sleep of waters theatrical presentation at her animated library exhibit wednesday at out north the battery gives me about hours usage under normal web browsing together with some office applications it really pisses me off submitted by mikel on sat we should also stop taxing businesses as individuals but rather reduce rates to which would help business to grow and create jobs ousley also won titles in breakaway roping and bradley in tiedown roping beyond that im not sure anybody agrees on anything be ready for a bunch of hostiles they were kind of targeting him and if youre not giving your best effort it allows the other team to feed off of that troy orion tom of beclabito killed aug by an improvised explosives device when his squad went to the aid of another that had come under heavy fire how many did it during the era we had the answers designed for those who recently lost a loved one he would like to provide more coverage to the uninsured by allowing people to buy into medicaid coverage an unknown suspect shot into a the residence and a vehicle that was parked in front of craig st neither the vehicle nor the residence was occupied at the time backtoback home runs by dollar ridgell and jerry powell highlighted an eightrun firstinning as morehouse went on to a convincing second round victory ron mcclelland the pastpresident of the and law association sees the changes as a case of jobs and services being moved out of the community ryan wittman has the game is upsidedown ahh well the lower prices just give myself and you more time to get more gold before it moves much higher so take it as a blessing in disguise shes the best candidate weve had since shandy finnessey but better earth day events include an art exhibition featuring kimberly piazzas tire recycling concepts at the pacific pinball museum in alameda and a screening of natures half acre about insects and flowers at the walt disney family museum in san francisco most of the paratroopers are still arriving trying to assess conditions and find the right local officials to work with the most lush lowkey spot in the world with a code attached to it she really comes across as a hypocrite steve teques was fifth in the the anniversary earth day will be observed thursday and the mesa republic is taking a look at four local businesses that use green practices days a year harry and nancy plastered them up at every press conference health care is legislation and the supposed divide of polls are within the margin of error of said pollsin which case there is no provable divide and the use journalistically of the word divide is inappropriate they would show about as much understanding of our system reagan agreed immediately it is a known fact what we arent paying out in decent shelter and living allowances we are paying out later in medical expenses prisons and child protection services we enter like animals and go out like animals says samir sublaban a grocery store owner he has missed straight games since suffering a torn groin at ottawa on jan and isnt expected to return until after the olympic break theyre not viable in commercial theaters without movie stars at least the government will then give you a tax break so you can buy a house cousins is taller than jefferson but should be able to put up similar doubledouble numbers cant even seem to sign up on atts website to check that means the tests are set to ignore the majority of banks holdings of sovereign debt as they moved the riskier elements of the debt out of their trading books so that it was left out of the scope of the tests compelling story lines involving the city of new orleans and itsongoing recovery from hurricane katrina and the attempt at a secondsuper bowl ring for indianapolis quarterback peyton manningpropelled the viewership they lost their mental strength after qualifying for the world cup strike while the irons hot unite says another was not sent a form and while their lives dont revolve solely around potter these days its still a source of good fun lynn neuenswander public information officer for the department of behavioral health part of the countys continuum of care said the face of homelessness is changing so please leave well enough alone unless you can do better or help in some way he said once a month he plans to open both sides both party central and party zone for larger parties juan uribe walked and sanchez doubled home sandoval loan from a bank or credit union more financial institutions are offering shortterm loans to people with poor credit with the mailout of census questionnaires slightly more than a month away the census bureau will run three ads promoting census awareness during the super bowl telecast two during the pregame show and one during the third quarter she is saying she is doing this for fun and why cant moms have fun rookie percy harvin is a real threat as is visanthe shiancoe tds including playoffs how are we to judge the austerity measures just passed by the merkel government ms chithra would not be eliminated or replaced by anybody think ill miss the walks to the stadium more than the games themselves its dans understanding and we have neurological evidence that abbie is not capable of seeing or having any cognitive understanding of the children even if they were standing in front of her greene said predicts a percent increase in the number of marylanders traveling by car this weekend in trying to find his best diego led argentina into battle on occasions with a record of luftha'

start = timmer.time()
HD_ngram_mod = ngram_encode_mod(str_tst, HD_aphabet_mod, aphabet_mod, n_size_mod) # HD_ngram is a projection of n-gram statistics for str to N-dimensional space. It can be used to learn the word embedding
end = timmer.time()
print(end - start) # get execution time

start = timmer.time()
HD_ngram2_mod = ngram_encode_hybrid(str_tst, HD_aphabet_mod, aphabet_mod, n_size_mod) # HD_ngram is a projection of n-gram statistics for str to N-dimensional space. It can be used to learn the word embedding
end = timmer.time()
print(end - start)  # get execution time

print(np.sum(np.abs(HD_ngram2_mod-HD_ngram_mod))) # check correctness of obtained HD vectors
print(HD_ngram2_mod)
print(HD_ngram_mod[:20])
